In [18]:
# Imports

sequence = [['PH2', 'HD2', 'HDR', 'HDA', 'HDE'],
            ['PH2', 'HD2', 'PH7', 'HD7', 'HDR'],
            ['PH7', 'HD7', 'HDR']]

In [19]:
# Tree data structure
class Tree():
    Item = None
    Parent = None
    Children = None

    def __init__(self, item_value=None):
        self.Item = item_value
        self.Children = []
        self.Parent = None

    def add_child(self, child):
        newchild = Tree(child)
        newchild.Parent = self
        self.Children.append(newchild)

    def get_child(self, target):
        for chld in self.Children:
            if chld.Item == target:
                return chld
        return None

    def get_children(self):
        return self.Children

    def has_child(self, target):
        found = self.get_child(target)
        if found is not None:
            return True
        else:
            return False

    def remove_child(self, child):
        for chld in self.Children:
            if chld.Item == child:
                self.Children.remove(chld)

In [20]:
# Training = Create prediction tree, LT and II
# Initialize variables
alphabet = set()
II = {}
LT = {}

# Root Node
root_node = Tree()

for idx, seq in enumerate(sequence):
    # Start from root node
    current_node = root_node
    for item in seq:
        # Update complete list of item used
        alphabet.add(item)
        # Add a new branch if not existing
        if not current_node.has_child(item):
            current_node.add_child(item)
        # Move one level down in tree
        current_node = current_node.get_child(item)

        # Create set in Inverted index if item not existing
        if II.get(item) is None:
            II[item] = set()
        # Add idx to II
        II[item].add(idx)

    # Add last item to the lookup table
    LT[idx] = current_node

In [21]:
t_seq = ['HD2','PH7']

In [22]:
# Predict
predictions = []
k = 3 # history depth?
n = 1 # nb of predictions

t_seq = t_seq[-k:] # Take only the last k element of sequence in target

all_seqs = set(range(0, len(sequence))) # TODO before loop

for item in t_seq:
    intersection = all_seqs & II.get(item)

similar_sequences = []
for element in intersection:
    similar_sequences.append(sequence[element])

counttable = {}

for seq in similar_sequences:
    index = next(i for i, v in zip(range(len(seq) - 1, 0, -1), reversed(seq)) if v == t_seq[-1])
    if index is not None:
        count = 1
        for element in seq[index + 1:]:
            if element in t_seq:
                continue
            weight_level = 1 / len(similar_sequences)
            weight_distance = 1 / count
            score = 1 + weight_level + weight_distance * 0.001

            if counttable.get(element) is None:
                counttable[element] = score
            else:
                counttable[element] = score * counttable.get(element)
            counttable = counttable

            count += 1

largest = sorted(counttable.items(), key=lambda t: t[1], reverse=True)[:n]
pred = [key for key, _ in largest]

predictions.append(pred)

StopIteration: 

In [23]:
predictions

[]

In [52]:
sequence = ['PH2', 'HD2', 'HDR', 'HDA', 'HDE']
t_seq = ['PH2','HDZ']

In [53]:
[i for i,v in zip(range(len(sequence) - 1, 0, -1), reversed(sequence)) if v == t_seq[-1]]

[]

In [54]:
sequence.index(t_seq[-1])

ValueError: 'HDZ' is not in list

In [36]:
t_seq[-1]

'HD2'

In [47]:
list(zip(range(len(sequence) - 1, 0, -1), reversed(sequence)))

[(4, 'HDE'), (3, 'HDA'), (2, 'HDR'), (1, 'HD2')]